In [52]:
from collections import Counter

class ThreeDChessRooks:
    def count(self, C, R, XP, YP, ZP, seed):
#         def print(*args):
#             pass
        
        pool = []
        
        for x,y,z in zip(XP, YP, ZP):
            pool.append((x,y,z))

        state = seed
        for i in range(len(XP), R):
            state = (state * 1103515245 + 12345) % 2147483648
            x = state%C
            state = (state * 1103515245 + 12345) % 2147483648
            y = state%C
            state = (state * 1103515245 + 12345) % 2147483648
            z = state%C
            pool.append((x,y,z))
            
        print(pool)
        res = 0

        single_match = Counter()
        
        for x,y,z in pool:
            single_match[x,-1,-1] += 1
            single_match[-1,y,-1] += 1
            single_match[-1,-1,z] += 1
        
        for k,v in single_match.items():
            res += v*(v-1)
        
        print(single_match)
        print("single_match res", res)
        
        double_match = Counter()
        for x,y,z in pool:
            double_match[x,y,-1] += 1
            double_match[-1,y,z] += 1
            double_match[x,-1,z] += 1
        
        for k,v in double_match.items():
            res -= v*(v-1)


            for k,v in double_match.items():
            x,y,z = k
            
            if x == -1:
                if (0,y,z) in pool and (1,y,z) in pool:
                    res -= 1
                if (C-1,y,z) in pool and (C-2,y,z) in pool:
                    res -= 1
                    
            if y == -1:
                if (x,0,z) in pool and (x,1,z) in pool:
                    res -= 1
                if (x,C-1,z) in pool and (x,C-2,z) in pool:
                    res -= 1
                    
            if z == -1:
                if (x,y,0) in pool and (x,y,1) in pool:
                    res -= 1
                if (x,y,C-1) in pool and (x,y,C-2) in pool:
                    res -= 1
        
        c = Counter(pool)
        for k,v in c.items():
            res -= v*(v-1)
        
        return res        

In [53]:
s = ThreeDChessRooks()

In [54]:
s.count(10,10,[9,2,3,4],[5,6,7,8],[9,0,1,9],47474747)

[(9, 5, 9), (2, 6, 0), (3, 7, 1), (4, 8, 9), (8, 1, 8), (5, 8, 7), (2, 1, 6), (3, 2, 5), (0, 9, 4), (3, 0, 7)]
Counter({(3, -1, -1): 3, (-1, -1, 9): 2, (2, -1, -1): 2, (-1, 8, -1): 2, (-1, 1, -1): 2, (-1, -1, 7): 2, (9, -1, -1): 1, (-1, 5, -1): 1, (-1, 6, -1): 1, (-1, -1, 0): 1, (-1, 7, -1): 1, (-1, -1, 1): 1, (4, -1, -1): 1, (8, -1, -1): 1, (-1, -1, 8): 1, (5, -1, -1): 1, (-1, -1, 6): 1, (-1, 2, -1): 1, (-1, -1, 5): 1, (0, -1, -1): 1, (-1, 9, -1): 1, (-1, -1, 4): 1, (-1, 0, -1): 1})


16

In [55]:
s.count(99,8,
        [47, 48, 49, 50, 50, 51, 52, 53],
        [12, 12, 12, 12, 12, 12, 12, 12],
        [12, 12, 12, 12, 12, 12, 12, 12], 111
       )

[(47, 12, 12), (48, 12, 12), (49, 12, 12), (50, 12, 12), (50, 12, 12), (51, 12, 12), (52, 12, 12), (53, 12, 12)]
Counter({(-1, 12, -1): 8, (-1, -1, 12): 8, (50, -1, -1): 2, (47, -1, -1): 1, (48, -1, -1): 1, (49, -1, -1): 1, (51, -1, -1): 1, (52, -1, -1): 1, (53, -1, -1): 1})


52